# This notebook covers Reinforcement learning algorithm i.e. Q Learning.

In [1]:
# Import NumPy and Random libraries.
import numpy as np
import random

In [2]:
# Define the gridworld environment.
# 'S' represents the starting point, 'G' represents the goal, 'H' represents
# a hole, and 'F' represents a regular empty cell.
# The agent can move in four directions: up, down, left, and right.

grid = np.array([
    ['S', 'F', 'F', 'F'],
    ['F', 'H', 'F', 'H'],
    ['F', 'F', 'F', 'H'],
    ['H', 'F', 'F', 'G']
])

print("Grid Environment:")
print(grid)
print()

Grid Environment:
[['S' 'F' 'F' 'F']
 ['F' 'H' 'F' 'H']
 ['F' 'F' 'F' 'H']
 ['H' 'F' 'F' 'G']]



In [3]:
# Define the rewards for each state.
rewards = {
    'G': 100, # Goal state
    'H': -100, # Hole state
    'S': 0, # Start state
    'F': -1 # Empty cell
}

In [4]:
# Define parameters.
learning_rate = 0.1
discount_factor = 0.9
num_episodes = 1000

In [5]:
# Initialize the Q-table with zeros
num_states = np.prod(np.shape(grid))
num_actions = 4 # Up, Down, Left, Right
Q = np.zeros((num_states, num_actions))

In [6]:
# Helper function to convert (row, col) to state index
def state_index(row, col):
    return row * len(grid[0]) + col

# Helper function to convert state index to (row, col)
def index_to_state(index):
    row = index // len(grid[0])
    col = index % len(grid[0])
    return row, col

# Epsilon-greedy action selection
def choose_action(state, epsilon=0.1):
    if random.random() < epsilon:
        return random.randint(0, num_actions - 1)  # Explore: random action
    else:
        return np.argmax(Q[state, :])  # Exploit: best action


In [7]:
# Q-learning algorithm.
episode_rewards = []

for episode in range(num_episodes):
    state = state_index(0, 0)  # Start from the top-left corner (S)
    done = False
    total_reward = 0
    steps = 0
    max_steps = 100  # Prevent infinite loops

    while not done and steps < max_steps:
        action = choose_action(state, epsilon=0.1)  # Using epsilon-greedy policy
        current_row, current_col = index_to_state(state)

        # Calculate next position based on action
        next_row, next_col = current_row, current_col

        if action == 0:  # Up
            next_row = max(0, current_row - 1)
        elif action == 1:  # Down
            next_row = min(len(grid) - 1, current_row + 1)
        elif action == 2:  # Left
            next_col = max(0, current_col - 1)
        elif action == 3:  # Right
            next_col = min(len(grid[0]) - 1, current_col + 1)

        next_state = state_index(next_row, next_col)
        reward = rewards[grid[next_row, next_col]]
        total_reward += reward

        # Q-learning update rule
        Q[state, action] += learning_rate * (
            reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action]
        )

        state = next_state
        steps += 1

        # Check if episode is done (reached goal or hole)
        if grid[next_row, next_col] == 'G' or grid[next_row, next_col] == 'H':
            done = True

    episode_rewards.append(total_reward)

    # Print progress every 100 episodes
    if (episode + 1) % 200 == 0:
        avg_reward = np.mean(episode_rewards[-100:])
        print(f"Episode {episode + 1}: Average reward (last 100): {avg_reward:.2f}")

print("\nTraining completed!")
print(f"Final average reward (last 100 episodes): {np.mean(episode_rewards[-100:]):.2f}")

print("\nQ-table:")
print(Q)


Episode 200: Average reward (last 100): -17.19
Episode 400: Average reward (last 100): -16.43
Episode 600: Average reward (last 100): -13.91
Episode 800: Average reward (last 100): 15.19
Episode 1000: Average reward (last 100): 53.06

Training completed!
Final average reward (last 100 episodes): 53.06

Q-table:
[[ 16.73064776  54.95380349  11.86613894   0.52595009]
 [ -0.99427358 -98.92247363   0.           7.36712881]
 [ -0.88976544  25.41907768  -0.92023356  -1.02309064]
 [ -0.95617925 -10.          -0.99792227  -0.95617925]
 [ 17.30648918  62.17097688  21.71908456 -99.75349653]
 [  0.           0.           0.           0.        ]
 [ -0.45816856  62.18239809 -27.1        -19.        ]
 [  0.           0.           0.           0.        ]
 [ 18.00130586 -68.61894039  28.28293263  70.18999384]
 [-61.2579511   16.12807992  37.50001803  79.09999846]
 [ 17.94466193  88.99999985  36.43930569 -61.2579511 ]
 [  0.           0.           0.           0.        ]
 [  0.           0.        

In [8]:
# Use the learned Q-table to find the optimal policy.
optimal_policy = []
action_names = ['Up', 'Down', 'Left', 'Right']

print("\nOptimal Policy:")
for row in range(len(grid)):
    policy_row = []
    for col in range(len(grid[0])):
        if grid[row, col] == 'H':
            policy_row.append('Hole')
        else:
            state = state_index(row, col)
            best_action = np.argmax(Q[state, :])
            policy_row.append(action_names[best_action])
    optimal_policy.append(policy_row)

optimal_policy = np.array(optimal_policy)
print(optimal_policy)


Optimal Policy:
[['Down' 'Right' 'Down' 'Up']
 ['Down' 'Hole' 'Down' 'Hole']
 ['Right' 'Right' 'Down' 'Hole']
 ['Hole' 'Right' 'Right' 'Up']]


In [9]:
# Test the learned policy
def test_policy(start_row=0, start_col=0, max_steps=20):
    print(f"\nTesting policy starting from ({start_row}, {start_col}):")
    row, col = start_row, start_col
    path = [(row, col)]

    for step in range(max_steps):
        print(f"Step {step + 1}: Position ({row}, {col}) - Cell: {grid[row, col]}")

        if grid[row, col] == 'G':
            print("Reached Goal!")
            break
        elif grid[row, col] == 'H':
            print("Fell into Hole!")
            break

        state = state_index(row, col)
        action = np.argmax(Q[state, :])

        # Move based on action
        if action == 0:  # Up
            row = max(0, row - 1)
        elif action == 1:  # Down
            row = min(len(grid) - 1, row + 1)
        elif action == 2:  # Left
            col = max(0, col - 1)
        elif action == 3:  # Right
            col = min(len(grid[0]) - 1, col + 1)

        path.append((row, col))
        print(f"Action: {action_names[action]} -> New position: ({row}, {col})")

    return path

# Test the learned policy
test_path = test_policy()
print(f"\nPath taken: {test_path}")


Testing policy starting from (0, 0):
Step 1: Position (0, 0) - Cell: S
Action: Down -> New position: (1, 0)
Step 2: Position (1, 0) - Cell: F
Action: Down -> New position: (2, 0)
Step 3: Position (2, 0) - Cell: F
Action: Right -> New position: (2, 1)
Step 4: Position (2, 1) - Cell: F
Action: Right -> New position: (2, 2)
Step 5: Position (2, 2) - Cell: F
Action: Down -> New position: (3, 2)
Step 6: Position (3, 2) - Cell: F
Action: Right -> New position: (3, 3)
Step 7: Position (3, 3) - Cell: G
Reached Goal!

Path taken: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3)]
